In [1]:
## need to cite original author (originally titled Vaex Demo)

In [2]:
import numpy as np
import ipywidgets as widgets
import bqplot as bq

In [3]:
## random data to plot
size = 10000
scale = 1.
scaleLocal = 20
np.random.seed(0)
x_data = np.arange(size)
y_data = np.cumsum(np.random.randn(size)  * scale) + np.random.randn(size) * scaleLocal

np.random.seed(1)
scaleLocal2 = 3
y_data2 = np.cumsum(np.random.randn(size)  * scale) + np.random.randn(size) * scaleLocal2

In [4]:
## create lines and figure
x_sc = bq.LinearScale()
y_sc = bq.LinearScale()
y_sc2 = bq.LinearScale()

ax_x = bq.Axis(label='X', scale=x_sc, grid_lines='solid')
ax_y = bq.Axis(label='Y', scale=y_sc, orientation='vertical', grid_lines='solid')
ax_y2 = bq.Axis(label='2nd Y', scale=y_sc2, orientation='vertical', side = 'right', visible = False,grid_lines='none')

line1 = bq.Lines(x=x_data, y=y_data, scales={'x': x_sc, 'y': y_sc} , colors=['blue'],display_legend = False, labels=['y1'],stroke_width = 1.0)

line2 = bq.Lines(x=x_data, y=y_data2, scales={'x': x_sc, 'y': y_sc} , colors=['darkgreen'],display_legend = False, labels=['y2'],stroke_width = 1.0)

margins = dict(top = 50, bottom=40, left=50, right=50)
fig = bq.Figure(marks = [line1,line2], axes=[ax_x, ax_y] , fig_margin = margins , animation_duration=1000)
fig.layout.width = '98%'
fig.layout.height = '400px' 
fig.title = 'Barplot'



In [7]:
class legendWidget(object):
    """A legend Widget using a horizontal bar chart
    
    marks: line marks from a bqplot figure.  
    
    These line marks must have legend labels 
    (in line mark, remove other legend by using this: display_legend = False)
    e.g. >>> legend = legendWidget(fig.marks) 
    
    BQplot module imported as bq (import bqplot as bq)

    """
    def __init__(self, lineFig):
        """Return a new Legend object."""
        
        self.lineFig = lineFig
        self.marks = self.lineFig.marks
        y_ord = bq.OrdinalScale()
        x_sc = bq.LinearScale()
        
        legendLabels = []
        colours = []
        markLineNums = [] # record number of lines per mark
        for mark in self.marks:            
            legendLabels += mark.labels
            colours += mark.colors[:len(mark.labels)]
            markLineNums.append(len(mark.labels))  
            
            

        bar = bq.Bars(
            y=[1]*len(legendLabels) , # all bars have a amplitude of 1
            x=legendLabels, 
            scales={'y': x_sc, 'x': y_ord},
            colors=colours ,
            padding = 0.6,
            orientation='horizontal',
            stroke = 'white'  #remove the black border around the bar
            )
        
        ax_y = bq.Axis(scale=y_ord, orientation="vertical")
        ax_x = bq.Axis(scale=x_sc)
        ax_x.visible = False
        margin = dict(top=40, bottom=0, left=110, right=5)
        barFig = bq.Figure(marks=[bar], axes=[ax_y, ax_x], fig_margin=margin)
        
        # Variable height depending on number of bars in legend
        barFig.layout.height = str(45 + 20 * len(legendLabels)) + 'px'
        barFig.layout.width = '170px'

        barFig.min_aspect_ratio = 0.000000000001 # effectively remove aspect ratio constraint
        barFig.max_aspect_ratio = 999999999999999 # effectively remove aspect ratio constraint
        barFig.background_style = {'fill': 'White'}   
                    
        self.fig = barFig
        self.bar = bar
        self.bar.on_element_click(self.changeOpacity)
        self.colours = colours
        self.markLineNums = markLineNums
        
    
    
    def matchLegendOpacities2FigMarks(self,bar):
            # Some marks in line plot have more than 1 line.  
        currentLineNum=0
        for markNum,markLineNum in enumerate(self.markLineNums):
            self.lineFig.marks[markNum].opacities = bar.opacities[currentLineNum:currentLineNum + markLineNum ]
            currentLineNum+=markLineNum

    def changeOpacity(self, bar, target):
        """
        Enable legend interactivity. 
        Use in conjunction with class legendWidget(object) 
        Click on legend bar to toggle opacity of all other lines
        """
      
        opacity = 0.1   # set opacity of non selected lines here
        sigNum = target['data']['index']
        if bar.opacities == []:
            bar.opacities=[1.0]*len(bar.x)

        if bar.opacities[sigNum] == 1.0:
            # bar.opacities[sigNum]=opacity    # Doesn't work ????
            bar.opacities=bar.opacities[:sigNum] + [opacity] + bar.opacities[sigNum+1:]
        else:
            bar.opacities=bar.opacities[:sigNum] + [1] + bar.opacities[sigNum+1:] 

        self.matchLegendOpacities2FigMarks(bar)


 
    
        

In [8]:
# legend = legendWidget(fig.marks)        
legend = legendWidget(fig)        
fig_legend = widgets.HBox([legend.fig,fig])
fig_legend